# [LVV-T2261](https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2261)
**Update Me**

Created by Bruno Quint, updated by Holger Drass

This notebook is to determine the rotator torques at different rotator angles with the camera hexapod at extreme positions of its space envelope.

This notebook is considered to be used once the refrigitation lines are not pressurezied and each time a paar of lines get pressurized. 

Use the "setup" notebook to control the state machine of the components.

Make 30 degree steps to allow the rotator to arrive to full speed = full torque value 

The analysis is done in a separate notebook.

## Start Up

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

/tmp/ipykernel_8287/1665379685.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) get_node. (Please use lsst.rsp.get_node())
  nb.utils.get_node()


'yagan07'

In [31]:
%matplotlib inline
import os
import sys
import asyncio
import logging

import numpy as np
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

import os
import pandas as pd

from astropy.time import Time
from scipy.signal import argrelextrema

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS

from datetime import datetime, timedelta

In [32]:
summit = 1 #use this for summit testing
# summit = 0 #use this for NCSA

## Check environment setup

The following cell will print some of the basic DDS configutions.

In [33]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ.get("LSST_DDS_DOMAIN_ID", "Expected, not set."))

file:///home/hdrass/WORK/ts_ddsconfig/config/ospl-shmem.xml
summit
0


### Setup logging

Setup logging in debug mode and create a logger to use on the notebook.

In [34]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [35]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

### Starting communication resources

We start by creating a domain and later instantiate the MTCS class.
We will use the class to startup the components. 

In [36]:
domain = salobj.Domain()

In [37]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [38]:
await mtcs.start_task

[None, None, None, None, None, None, None, None, None, None]

In [39]:
script = salobj.Controller("Script", index=42658886)
await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too

In [40]:
await script.start_task

In [67]:
await mtcs.rem.mtrotator.cmd_move.set_start(position=0, timeout=5.)


# Test start

## Iteration 1: Move the hexapod to (X=0, Y=0, Z=0, U=0, V=0, W=0)

In [41]:
now = datetime.now()
test_message = "LVV-2261: Hexapod at 0 - Pathfinder Lines Not Pressurized"
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=0, v=0, w=0, sync=True)
print(f" Done.")

 Done.


In [42]:
for p in [0, 30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

 2022-04-06 15:49:20.202762 - Moving to 0
 Done.
 2022-04-06 15:49:40.292051 - Moving to 30
 Done.
 2022-04-06 15:50:00.348643 - Moving to 60
 Done.
 2022-04-06 15:50:20.453894 - Moving to 90
 Done.
 2022-04-06 15:50:40.560005 - Moving to 60
 Done.
 2022-04-06 15:51:00.665760 - Moving to 30
 Done.
 2022-04-06 15:51:20.721903 - Moving to 0
 Done.
 2022-04-06 15:51:40.827490 - Moving to -30
 Done.
 2022-04-06 15:52:00.933018 - Moving to -60
 Done.
 2022-04-06 15:52:21.039654 - Moving to -90
 Done.
 2022-04-06 15:52:41.094990 - Moving to -60
 Done.
 2022-04-06 15:53:01.200554 - Moving to 0
 Done.


In [43]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 2: Move the hexapod to (X=0, Y=0, Z=0, U=-0.36 deg, V=0, W=0)

In [44]:
test_message = "LVV-2261: Hexapod u=-0.36deg - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
#Moving the hexapod in smaller steps here at the beginning of the test to make sure no cooling line gets damaged.
for _u in [-0.08, -0.16, -0.32, -0.36]:
    print(f" {datetime.now()} - Moving Hexapod to u={_u}")
    await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=_u, v=0, w=0, sync=True)
    await asyncio.sleep(15.)  # Just so if we need to press the e-stops
    print(f" Done.")

 2022-04-06 15:53:21.376141 - Moving Hexapod to u=-0.08
 Done.
 2022-04-06 15:53:36.463537 - Moving Hexapod to u=-0.16
 Done.
 2022-04-06 15:53:51.570125 - Moving Hexapod to u=-0.32
 Done.
 2022-04-06 15:54:06.677618 - Moving Hexapod to u=-0.36
 Done.


In [45]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

 2022-04-06 15:54:37.357539 - Moving to 30
 Done.
 2022-04-06 15:54:57.434520 - Moving to 60
 Done.
 2022-04-06 15:55:17.490191 - Moving to 90
 Done.
 2022-04-06 15:55:37.596421 - Moving to 60
 Done.
 2022-04-06 15:55:57.702237 - Moving to 30
 Done.
 2022-04-06 15:56:17.808795 - Moving to 0
 Done.
 2022-04-06 15:56:37.914161 - Moving to -30
 Done.
 2022-04-06 15:56:57.969306 - Moving to -60
 Done.
 2022-04-06 15:57:18.076033 - Moving to -90
 Done.
 2022-04-06 15:57:38.182385 - Moving to -60
 Done.
 2022-04-06 15:57:58.287212 - Moving to 0
 Done.


In [46]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 3: u=-0.253 v=-0.253

In [47]:
test_message = "LVV-2261: Hexapod u=-0.253deg v=-0.253deg (total 0.36) - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=-0.253, v=-0.253, w=0, sync=True)
await asyncio.sleep(15.)  # Just so if we need to press the e-stops
print(f" Done.")

 Done.


In [48]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

 2022-04-06 15:58:33.547900 - Moving to 30
 Done.
 2022-04-06 15:58:53.653723 - Moving to 60
 Done.
 2022-04-06 15:59:13.709032 - Moving to 90
 Done.
 2022-04-06 15:59:33.815262 - Moving to 60
 Done.
 2022-04-06 15:59:53.919843 - Moving to 30
 Done.
 2022-04-06 16:00:14.026642 - Moving to 0
 Done.
 2022-04-06 16:00:34.081928 - Moving to -30
 Done.
 2022-04-06 16:00:54.188284 - Moving to -60
 Done.
 2022-04-06 16:01:14.293063 - Moving to -90
 Done.
 2022-04-06 16:01:34.399911 - Moving to -60
 Done.
 2022-04-06 16:01:54.505836 - Moving to 0
 Done.


In [49]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 4: u=0.36 v=0.00

In [50]:
test_message = "LVV-2261: Hexapod u=+0.36deg v=0.00deg - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")

for _v in [-0.25, -0.14, 0.00]:
    print(f" {datetime.now()} - Moving Hexapod to v={_v}")
    await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=-0.25, v=_v, w=0, sync=True)
    await asyncio.sleep(15.)  # Just so if we need to press the e-stops
    print(f" Done.")
    
for _u in [-0.25, -0.14, 0.00, 0.08, 0.16, 0.32, 0.36]:
    print(f" {datetime.now()} - Moving Hexapod to u={_u}")
    await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=_u, v=0, w=0, sync=True)
    await asyncio.sleep(15.)  # Just so if we need to press the e-stops
    print(f" Done.")

 2022-04-06 16:02:14.638701 - Moving Hexapod to v=-0.25
 Done.
 2022-04-06 16:02:29.776275 - Moving Hexapod to v=-0.14
 Done.
 2022-04-06 16:02:44.884988 - Moving Hexapod to v=0.0
 Done.
 2022-04-06 16:02:59.993426 - Moving Hexapod to u=-0.25
 Done.
 2022-04-06 16:03:15.100209 - Moving Hexapod to u=-0.14
 Done.
 2022-04-06 16:03:30.209244 - Moving Hexapod to u=0.0
 Done.
 2022-04-06 16:03:45.317878 - Moving Hexapod to u=0.08
 Done.
 2022-04-06 16:04:00.428274 - Moving Hexapod to u=0.16
 Done.
 2022-04-06 16:04:15.533992 - Moving Hexapod to u=0.32
 Done.
 2022-04-06 16:04:30.691079 - Moving Hexapod to u=0.36
 Done.


In [53]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

 2022-04-06 16:11:40.082514 - Moving to 30
 Done.
 2022-04-06 16:12:00.178685 - Moving to 60
 Done.
 2022-04-06 16:12:20.283675 - Moving to 90
 Done.
 2022-04-06 16:12:40.339037 - Moving to 60
 Done.
 2022-04-06 16:13:00.445729 - Moving to 30
 Done.
 2022-04-06 16:13:20.551311 - Moving to 0
 Done.
 2022-04-06 16:13:40.656854 - Moving to -30
 Done.
 2022-04-06 16:14:00.713074 - Moving to -60
 Done.
 2022-04-06 16:14:20.819269 - Moving to -90
 Done.
 2022-04-06 16:14:40.924462 - Moving to -60
 Done.
 2022-04-06 16:15:01.029433 - Moving to 0
 Done.


In [54]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 5: u=-0.253deg v=+0.253deg

In [55]:
test_message = "LVV-2261: Hexapod u=-0.253deg v=+0.253deg (total 0.36) - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")

for _u in [0.36, 0.253, 0.12, 0, -0.12, -0.253]:
    print(f" {datetime.now()} - Moving Hexapod to u={_u}")
    await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=_u, v=0, w=0, sync=True)
    await asyncio.sleep(15.)  # Just so if we need to press the e-stops
    print(f" Done.")
    
for _v in [0.12, 0.253]:
    print(f" {datetime.now()} - Moving Hexapod to v={_v}")
    await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=-0.253, v=_v, w=0, sync=True)
    await asyncio.sleep(15.)  # Just so if we need to press the e-stops
    print(f" Done.")

 2022-04-06 16:15:21.158556 - Moving Hexapod to u=0.36
 Done.
 2022-04-06 16:15:36.267707 - Moving Hexapod to u=0.253
 Done.
 2022-04-06 16:15:51.374781 - Moving Hexapod to u=0.12
 Done.
 2022-04-06 16:16:06.484201 - Moving Hexapod to u=0
 Done.
 2022-04-06 16:16:21.590529 - Moving Hexapod to u=-0.12
 Done.
 2022-04-06 16:16:36.697551 - Moving Hexapod to u=-0.253
 Done.
 2022-04-06 16:16:51.805295 - Moving Hexapod to v=0.12
 Done.
 2022-04-06 16:17:06.912511 - Moving Hexapod to v=0.253
 Done.


In [56]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

 2022-04-06 16:17:22.052280 - Moving to 30
 Done.
 2022-04-06 16:17:42.176598 - Moving to 60
 Done.
 2022-04-06 16:18:02.232126 - Moving to 90
 Done.
 2022-04-06 16:18:22.337693 - Moving to 60
 Done.
 2022-04-06 16:18:42.443660 - Moving to 30
 Done.
 2022-04-06 16:19:02.549905 - Moving to 0
 Done.
 2022-04-06 16:19:22.654915 - Moving to -30
 Done.
 2022-04-06 16:19:42.709657 - Moving to -60
 Done.
 2022-04-06 16:20:02.815960 - Moving to -90
 Done.
 2022-04-06 16:20:22.921748 - Moving to -60
 Done.
 2022-04-06 16:20:43.028266 - Moving to 0
 Done.


In [57]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 6: Move the hexapod to (X=0, Y=0, Z=0, U=+0.253deg, V=+0.253deg, W=0)

In [58]:
test_message = "LVV-2261: Hexapod x=0, Y=0, Z=0,U=+0.253deg, V=+0.253deg, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  x=0, Y=0, Z=0, U=+0.253deg, V=+0.253deg, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=+0.253, v=+0.253, w=0, sync=True)
print(f" Done.")

 2022-04-06 16:21:03.157312 - Moving Hexapod to  x=0, Y=0, Z=0, U=+0.253deg, V=+0.253deg, W=0
 Done.


In [63]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

 2022-04-06 16:40:08.402547 - Moving to 30
 Done.
 2022-04-06 16:40:28.475600 - Moving to 60
 Done.
 2022-04-06 16:40:48.580639 - Moving to 90
 Done.
 2022-04-06 16:41:08.636323 - Moving to 60
 Done.
 2022-04-06 16:41:28.738926 - Moving to 30
 Done.
 2022-04-06 16:41:48.844599 - Moving to 0
 Done.
 2022-04-06 16:42:08.951936 - Moving to -30
 Done.
 2022-04-06 16:42:29.053671 - Moving to -60
 Done.
 2022-04-06 16:42:49.158638 - Moving to -90
 Done.
 2022-04-06 16:43:09.262553 - Moving to -60
 Done.
 2022-04-06 16:43:29.366956 - Moving to 0
 Done.


In [64]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 7: Move the hexapod to (X=0, Y=0, Z=0, U=+0.253deg, V=-0.253deg, W=0)

In [65]:
test_message = "LVV-2261: Hexapod x=0, Y=0, Z=0,U=+0.253deg, V=-0.253deg, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  x=0, Y=0, Z=0, U=+0.253deg, V=-0.253deg, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=+0.253, v=-0.253, w=0, sync=True)
print(f" Done.")

 2022-04-06 16:43:49.541650 - Moving Hexapod to  x=0, Y=0, Z=0, U=+0.253deg, V=-0.253deg, W=0
 Done.


In [68]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=10.)
    await asyncio.sleep(30.)
    print(f" Done.")

 2022-04-06 16:47:10.807238 - Moving to 30
 Done.
 2022-04-06 16:47:40.874724 - Moving to 60
 Done.
 2022-04-06 16:48:10.980953 - Moving to 90
 Done.
 2022-04-06 16:48:41.087779 - Moving to 60
 Done.
 2022-04-06 16:49:11.194375 - Moving to 30
 Done.
 2022-04-06 16:49:41.302015 - Moving to 0
 Done.
 2022-04-06 16:50:11.358480 - Moving to -30
 Done.
 2022-04-06 16:50:41.415234 - Moving to -60
 Done.
 2022-04-06 16:51:11.523812 - Moving to -90
 Done.
 2022-04-06 16:51:41.629452 - Moving to -60
 Done.
 2022-04-06 16:52:11.736494 - Moving to 0
 Done.


In [69]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 8: Move the hexapod to (X=0, Y=0, Z=0, U=0deg, V=0.36deg, W=0)

In [70]:
test_message = "LVV-2261: Hexapod x=0, Y=0, Z=0,U=0, V=+0.36deg, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  x=0, Y=0, Z=0, U=0, V=0.36deg, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=0, v=0.36, w=0, sync=True)
print(f" Done.")

 2022-04-06 16:52:41.913918 - Moving Hexapod to  x=0, Y=0, Z=0, U=0, V=0.36deg, W=0
 Done.


In [71]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

 2022-04-06 16:52:42.039817 - Moving to 30
 Done.
 2022-04-06 16:53:02.150392 - Moving to 60
 Done.
 2022-04-06 16:53:22.256442 - Moving to 90
 Done.
 2022-04-06 16:53:42.360682 - Moving to 60
 Done.
 2022-04-06 16:54:02.465870 - Moving to 30
 Done.
 2022-04-06 16:54:22.570725 - Moving to 0
 Done.
 2022-04-06 16:54:42.673857 - Moving to -30
 Done.
 2022-04-06 16:55:02.779259 - Moving to -60
 Done.
 2022-04-06 16:55:22.834112 - Moving to -90
 Done.
 2022-04-06 16:55:42.939412 - Moving to -60
 Done.
 2022-04-06 16:56:03.043266 - Moving to 0
 Done.


In [72]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 9: u=0.0deg v=-0.36deg 

In [73]:
test_message = "LVV-2261: Hexapod u=+0deg v=-0.36deg (total 0.36) - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to u=+0deg v=-0.36deg")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=0, v=0.36, w=0, sync=True)
print(f" Done.")

 2022-04-06 16:56:23.215713 - Moving Hexapod to u=+0deg v=-0.36deg
 Done.


In [76]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=10.)
    await asyncio.sleep(30.)
    print(f" Done.")

 2022-04-06 17:08:29.811195 - Moving to 30
 Done.
 2022-04-06 17:08:59.871448 - Moving to 60
 Done.
 2022-04-06 17:09:29.978339 - Moving to 90
 Done.
 2022-04-06 17:10:00.084639 - Moving to 60
 Done.
 2022-04-06 17:10:30.191964 - Moving to 30
 Done.
 2022-04-06 17:11:00.298666 - Moving to 0
 Done.
 2022-04-06 17:11:30.405572 - Moving to -30
 Done.
 2022-04-06 17:12:00.512763 - Moving to -60


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1009755505, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Not connected to the low-level controller.')

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 10: x=-11400, Y=0, Z=-10000

In [ ]:
test_message = "LVV-2261: Hexapod x=-11400, Y=0, Z=-10000 - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")

print(f" {datetime.now()} - Moving Hexapod to x=-11400, Y=0, Z=-10000")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=-11400, y=0, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=10.)
    await asyncio.sleep(30.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 11 = Move the hexapod to (X=11400, Y=0, Z=-10000, U=0, V=0, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod X=11400, Y=0, Z=-10000, U=0, V=0, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  X=11400, Y=0, Z=-10000, U=0, V=0, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=11400, y=0, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=10.)
    await asyncio.sleep(30.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 12: Move the hexapod to (X=6700, Y=6700, Z=-10000, U=0, V=0, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod X=6700, Y=6700, Z=-10000, U=0, V=0, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  X=6700, Y=6700, Z=-10000, U=0, V=0, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=6700, y=6700, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=10.)
    await asyncio.sleep(30.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 13: Move the hexapod to (X=6700, Y=-6700, Z=-10000, U=0, V=0, W=0

In [ ]:
test_message = "LVV-2261: Hexapod X=6700, Y=-6700, Z=-10000, U=0, V=0, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  X=6700, Y=-6700, Z=-10000, U=0, V=0, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=6700, y=-6700, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=10.)
    await asyncio.sleep(30.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 14: Move the hexapod to (X=-6700, Y=-6700, Z=-10000, U=0, V=0, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod X=-6700, Y=-6700, Z=-10000, U=0, V=0, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  X=-6700, Y=-6700, Z=-10000, U=0, V=0, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=-6700, y=-6700, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=10.)
    await asyncio.sleep(30.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 16: Move the hexapod to (X=0, Y=-11400, Z=-10000, U=0, V=0, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod X=0, Y=-10000, Z=-10000, U=0, V=0, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  X=0, Y=-10000, Z=-10000, U=0, V=0, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=-10000, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=10.)
    await asyncio.sleep(30.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 17: Move the hexapod to (X=0, Y=11400, Z=-10000, U=0, V=0, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod X=0, Y=11400, Z=-10000, U=0, V=0, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  X=0, Y=11400, Z=-10000, U=0, V=0, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=11400, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=10.)
    await asyncio.sleep(30.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")